# Latent Dirichlet Allocation

## Introduction

La Latent Dirichlet Allocation est un "modèle statistique génératif" : c'est une méthode qui permet de générer des groupes qui permettent d'expliquer statistiquement certaines ressemblances dans un ensemble de données. Utilisée comme méthode de Natural Language Processing, la LDA permet de modéliser des thèmes récurrents dans ces textes (*topic modelling* en anglais). En pratique, utiliser la LDA permet de :
- modéliser le corpus d'entrée de manière différente de BOW et Word2Vec
- générer des thèmes du corpus de texte (un thème = un ensemble de mots reliés sémantiquement au même sujet)

<img src="TopicModelling.png" width=600>

On va donc d'abord [comprendre la théorie](#sec1.), puis [s'intéresser à son application pratique](#sec2.), et enfin [comparer ces résultats](#sec3.) avec une autre méthode de *topic modelling* : la Nonnegative Matrix Factorization.

Sommaire :

* I\. [En théorie](#sec1.)<br>
    * 1. [But de la méthode](#sec1.1.)<br>
    * 2. [TLDR](#sec1.2.)<br>
    * 3. [Quelques détails](#sec1.3.)<br>
* II\. [En pratique](#sec2.)<br>
    * 1. [Récupération des données](#sec2.1.)<br>
    * 2. [BOW et TFIDF](#sec2.2.)<br>
    * 3. [Latent Dirichlet Allocation](#sec2.3.)<br>
    * 4. [Comparaison des représentations](#sec2.4.)<br>
    * 5. [Une autre méthode de Topic Modelling : la NMF](#sec2.5.)<br>

## <a id="sec1."></a> 1. En théorie

La mise en place de la LDA nécessite des notations de statistique bayésienne assez lourdes. On donne ici une présentation simple pour comprendre grossièrement le principe et les notations.

### <a id="sec1.1."></a> 1.1. But de la méthode
Le but de la méthode est de pouvoir associer à chaque mot de chaque document la probabilité qu'il soit issu d'un thème.

### <a id="sec1.2."></a> 1.2. TLDR
Pour résumer le principe de la méthode en quelques lignes :

- La distribution a priori des thèmes au sein de chaque document est une loi de Dirichlet
- Ensuite, pour chaque mot du corpus, on met à jour la loi a posteriori (i.e. "au vu des données") en fixant les distributions de tous les autres mots et en calculant les nouvelles distributions du mot considéré via formule de Bayes et intégrations. 

Il y a plusieurs distributions à considérer : la probabilité qu'un document soit assigné à un thème, qu'un mot soit issu d'un thème... Les calculs sont difficiles et nécessitent des approximations fines.

Cela permet aussi de comprendre le nom de la méthode : "Dirichlet" car c'est la distribution a priori thème-document et "Latent" car on réalise les calculs dans l'"espace des thèmes".

### <a id="sec1.3."></a> 1.3. Quelques détails
Quelques détails histoire de retirer l'aspect "boîte noire" (non-nécessaires pour la mise en pratique) : 
### Initialisation
On choisit d'abord le nombre de thèmes à générer, noté $T$. Pour chaque document $d \in \{1, ..., D\}$ du corpus, on génère $\theta_d \sim Dir(\alpha)$ (loi de Dirichlet) avec $\alpha \in ]0, 1[^T$ : 

- Chaque élément $\alpha_t$ de $\alpha$ correspond au poids a priori du thème t dans un document quelconque.
- La loi de Dirichlet est une sorte de "loi bêta en N-dimensions". C'est un choix qui permet de faciliter les calculs : cette distribution est conjuguée* à la loi multinomiale (binomial en N-dimension) qui est elle-même utilisée dans les calculs. C'est cette loi qui donne son nom à la méthode.

<img src="Dirichlet.png" width=600>
<center><b>Exemple de distribution de Dirichlet (3D)</b></center>

- $\theta_d \in [0, 1]^T$ représente les probabilités que chaque thème t apparaisse dans le document d. 

D'autre part, on fixe $\beta \in \mathcal{M}_{T, N}([0, 1])$ où N est le nombre total de mots : $\beta_{i, j} = \mathbb{P}$("Le mot d'indice j est issu du thème d'indice i"). Le but est ainsi d'estimer $\beta$.

\*sans rentrer dans le détail, une loi a priori est conjuguée à la loi du modèle expérimental ("vraisemblance") si la loi a posteriori (i.e. "au vu des données") a la même forme que la loi a priori. Il n'y a donc aucun calcul à réaliser dans ce cas !

### Apprentissage
Les probabilités ainsi initialisées sont complètement aléatoires. On améliore petit à petit le modèle jusqu'à stabilisation des distributions. Les notations sont simplifiées (presque erronées...) à but pédagogique.

Pour chaque mot $m$, chaque document $d$ et chaque thème $t$, on calcule :

- $p(t\mid d)$ la probabilité que le document $d$ soit assigné au thème $t$
- $p(w\mid t)$ la probabilité que le thème $t$ soit assigné au mot $w$

On peut alors calculer la probabilité que le thème $t$ génère $w$ dans le document $d$ via une intégrale du produit de ces probabilités. Il faut noter que ce dernier calcul (l'inférence) est très loin d'être trivial et nécessite des méthodes de calcul assez lourdes pour obtenir une bonne approximation (souvent par méthode "variationnelle bayésienne", pour citer son nom).

Une fois que les distributions sont stables, chaque thème est constitué des mots dont la probabilité d'être d'en être issu est parmi les plus fortes : un mot peut donc appartenir à plusieurs thèmes.

Pour plus de détails, le papier originel explique les différents calculs de probabilités : http://ai.stanford.edu/~ang/papers/jair03-lda.pdf

## <a id="sec2."></a> 2. En pratique

La LDA s'utilise dans un contexte de NLP. Pour suivre le fil rouge de NLP *spam or ham*, on peut utiliser la LDA pour :

- modéliser le corpus d'entrée de manière différente de BOW et Word2Vec et comparer les résultats de classification
- observer les thèmes récurrents dans les spams


### <a id="sec2.1."></a> 2. 1. Récupération des données

D'abord, on va chercher les données :

In [1]:
import os
import numpy as np

def get_emails(train_dir):
    email_path = []
    email_label = []
    for d in os.listdir(train_dir):
        folder = os.path.join(train_dir,d)
        email_path += [os.path.join(folder,f) for f in os.listdir(folder)]
        email_label += [f[0:3]=='spm' for f in os.listdir(folder)]
    return email_path, email_label

train_dir = '../data/lingspam_public/bare/' # Mettre le chemin correct s'il ne fonctionne pas
email_path, email_label = get_emails(train_dir)

print(f"Petit rappel : {len(email_path)} emails dont {np.sum(email_label)} spams")

Petit rappel : 2893 emails dont 481 spams


On crée un tokenizer qui retire tous les mots inutiles à la classification :

In [2]:
from nltk import wordpunct_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import words
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

class LemmaTokenizer(object):
    def __init__(self, stop_words = None, remove_non_words=True):
        self.wnl = WordNetLemmatizer()
        if stop_words is None:
            self.stopwords = set(stopwords.words('english'))
        else:
            self.stopwords = stop_words
        self.words = set(words.words())
        self.remove_non_words = remove_non_words
    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove non words
        if(self.remove_non_words):
            word_list = [word for word in word_list if word in self.words]
        # remove 1-character words
        word_list = [word for word in word_list if len(word)>1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [self.wnl.lemmatize(t) for t in word_list]
    
#L'outil qui "compte les mots"
countvect = CountVectorizer(input='filename',tokenizer=LemmaTokenizer(remove_non_words=True))

### <a id="sec2.2."></a> 2. 2. BOW et TFIDF

On obtient la représentation Bag Of Words correspondante (avec TFIDF) :

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer

#Petit rappel : le BOW est une matrice de taille (nb_documents, nb_mots)
bow = countvect.fit_transform(email_path)
#On relie la position dans le BOW au vrai mot grâce au dictionnaire
feat2word = {v: k for k, v in countvect.vocabulary_.items()}
#On pondère avec TFIDF (cf le cours sur le NLP)
X_tfidf = TfidfTransformer().fit_transform(bow)

### <a id="sec2.3."></a> 2. 3. Latent Dirichlet Allocation

En pratique, on réalise la LDA à partir du BOW. La transformation TFIDF ne donne en pratique pas de bons résultats car la division par la "fréquence de document" retire une information essentielle pour la LDA. On peut en revanche pondérer par la TF. En supposant que l'on veut déterminer 4 thèmes dans tous les emails :

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 3
tf = TfidfTransformer(use_idf = False).fit_transform(bow)
lda = LatentDirichletAllocation(n_components=n_topics, #nombre de thèmes, T précedemment
                                doc_topic_prior=None,  #précédemment alpha, par défaut 1/n_topics
                                topic_word_prior=None, #précédemment beta_i,j, par défaut 1/n_topics
                                max_iter=5, learning_method='online')
lda.fit(tf);

On peut alors afficher les mots du corpus qui correspondent le mieux à chaque thème obtenu :

In [9]:
def print_topics(model, feature_names, n_words):
    '''Affiche les n_words mots les plus probables pour chaque thème pour le modèle donné (LDA... ou NMF!)'''
    #topic_idx = 0, 1, 2, 3...
    #topic est la liste des probas que chaque mot soit issu du thème noté "topic_idx"
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        #topic.argsort()[:-n_words - 1:-1]] simply gives the indexes of most likely words for topic
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_words - 1:-1]])
        print(message)
    return

feature_names = countvect.get_feature_names()
print_topics(lda, feature_names, 15)

Topic #0: cloven crossword cleaver cleave divide hoof meat adhere yeni photography subject psoriasis sculpture tantalize tempt
Topic #1: cream syrup subject squash dope precinct fluff relish rumble collard bagel baked muffin tuna chipmunk
Topic #2: university language subject linguistics information mail one please de conference new would address research also


Bien que les thèmes sont a priori "théoriques", on peut retrouver une sémantique à travers ces thèmes (pas forcément dans cet ordre car l'ordre de création des thèmes est aléatoire) : 

- le thème 0 regroupe des termes autour du clivage (cloven, cleaver, divide, adhere...)
- le thème 1 regroupe des termes autour du langage et de l'académique (university, language, subject, information, research...)
- le thème 2 regroupe des termes autour de la cuisine (cream, syrup, sauce, cranberry, pantry, whipped...)

La représentation correspondante d'un document est donc un vecteur de taille n_topics dont chaque composante correspond à la probabilité d'être généré par un des thèmes :

In [15]:
X_lda = lda.fit_transform(tf)
print(f"Avec {tf.shape[0]} mails et {n_topics} thèmes, les emails sont représentés par une matrice de taille {X_lda.shape}")
print(f"Le premier document est représenté par {X_lda[0, :]}")

Avec 2893 mails et 3 thèmes, les emails sont représentés par une matrice de taille (2893, 3)
Le premier document est représenté par [0.05064384 0.89870755 0.05064861]


### <a id="sec2.4."></a> 2.4. Comparaison des représentations
On peut alors comparer les performances de ces représentations vis-à-vis d'une classification. On utilise par exemple les random forests pour l'étape de classification. D'abord TFIDF :

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

clf = RandomForestClassifier(n_estimators=100, criterion='entropy')
n_splits = 4

#Validation croisée stratifiée (on conserve les proportions de classe)
cv = StratifiedKFold(n_splits = n_splits, shuffle=True)

#Scores f1 en validation croisée (plus pertinent que la précision lorsque le dataset n'est pas équilibré)
scores = cross_val_score(clf, X_tfidf, email_label, cv=cv, scoring = 'f1')

print(f"Score F1 des Random Forests avec TFIDF : {scores.mean() : .2f} (+/- {scores.std() * 2 : .2f})")


Score F1 des Random Forests avec TFIDF :  0.93 (+/-  0.02)


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

clf = RandomForestClassifier(n_estimators=100, criterion='entropy')
n_splits = 4

#Validation croisée stratifiée (on conserve les proportions de classe)
cv = StratifiedKFold(n_splits = n_splits, shuffle=True)

#Scores f1 en validation croisée (plus pertinent que la précision lorsque le dataset n'est pas équilibré)
scores = cross_val_score(clf, X_lda, email_label, cv=cv, scoring = 'f1')

print(f"Score F1 des Random Forests avec LDA ({n_topics} thèmes) : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Score F1 des Random Forests avec LDA : 0.22 (+/- 0.05)


Sans grande surprise, on voit que la représentation TFIDF est de loin la meilleure pour la classification de spam. Les piètres performances de la LDA s'expliquent aisément : on tente dans ce cas de déterminer si un e-mail est un spam à partir de regroupements sémantiques; or un thème particulier peut aussi bien apparaître dans un mail normal que dans un spam (par exemple la "cuisine", thème observé précédemment). Même en augmentant le nombre de thèmes, on n'obtient pas de meilleurs résultats.

Plus généralement, la LDA est efficace lorsque ces groupes créés permettent d'expliquer la variable observée :

- En Biologie, cette méthode permet de détecter la présence d'une variation génétique *structurelle* au sein d'un groupe d'individus
- En Natural Language Processing, la LDA est souvent efficace pour la Sentiment Analysis : le regroupement par thèmes permet de déterminer si un texte est plutôt "positif" ou "négatif".

### <a id="sec2.5."></a> 2.5. Une autre méthode de Topic Modelling : la NMF

Sans rentrer dans le détail, il existe d'autres méthodes de Topic Modelling. On présente ici en pratique  la Nonnegative Matrix Factorization. Il s'agit initialement d'une méthode de factorisation matricielle permettant la décomposition d'une matrice quelconque V :
$$V = W \times H $$
où H et W ont des coefficients uniquements positifs.
<img src="NMF.jpg" width=600>
<center><b>Décomposition approximative NMF</b></center>

Cette méthode a des propriétés inhérentes de partitionnement : 
- en imposant H orthogonal, la méthode a une formulation mathématique équivalente à la méthode "K-means"
- avec une métrique particulière (divergence de Kullback–Leibler, très utilisée en statistique bayésienne), elle correspond à la méthode nommée pLSA, méthode dont est issue la LDA.

En appliquant cette factorisation à la matrice TFIDF, on peut donner un sens à W et H. On nomme T la dimension "latente" :
- V est de dimensions (D, N) : D documents pour N mots au total
- W est de dimensions (D, T) : D documents pour T "thèmes". W relie donc les documents aux "thèmes" latents.
- H est de dimensions (T, N) : T thèmes pour N mots. H relie donc les thèmes aux mots.
(consulter [ce lien](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) pour plus de détails)

On peut observer ce que cela donne sur les emails :

In [44]:
from sklearn.decomposition import NMF

n_topics = 3
nmf = NMF(n_components=n_topics).fit(bow)
feature_names = countvect.get_feature_names()
print_topics(nmf, feature_names, 15)

Topic #0: report mail program money people order make send name get work business time one address
Topic #1: language university conference linguistics de information session workshop research one may subject linguistic paper new
Topic #2: link directory web free net page search mail order world index add business offer best


On remarque que cette méthode donne des thèmes sensiblement différents de ceux obtenus avec LDA (bien que l'on retrouve un thème sur le langage et l'académique). Lors de l'utilisation de la LDA, on pourra donc comparer ses résultats avec ceux de la NMF.

# <a id="secconclusion"></a>Conclusion

Après avoir expliqué théoriquement la Latent Dirichlet Allocation, on en a présenté une application pratique sur le cas fil rouge *spam or ham*. Ce cas de classification est peu propice à la LDA car n'utilise pas correctement le partitionnement en "thèmes". En revanche, la LDA est très efficace lorsque les "thèmes" expliquent correctement la variable cible, comme en Sentiment Analysis par exemple. On a ensuite rapidement décrit la méthode Nonnegative Matrix Factorization comme alternative à la LDA : les thèmes obtenus sont différents de ceux générés par LDA et on pourra donc comparer les résultats de la NMF avec ceux de la LDA.